In [4]:
import os
import cv2
import numpy as np
from PIL import Image


def mask_function(input_path, gt_path):
    try:
        # Load and preprocess images
        input_image = Image.open(input_path).convert("RGB")  # Ensure RGB format
        input_image_np = np.array(input_image)
        gt_image_gray = Image.open(gt_path).convert("L")  # Load mask as grayscale
        gt_image_gray_np = np.array(gt_image_gray)

        # Convert input_image_np to grayscale
        input_image_gray_np = cv2.cvtColor(input_image_np, cv2.COLOR_RGB2GRAY)

        # Compute the difference
        difference = cv2.absdiff(gt_image_gray_np, input_image_gray_np)

        # Threshold the difference to create a binary mask
        _, binary_difference = cv2.threshold(difference, 1, 255, cv2.THRESH_BINARY)

        # Remove small noise with morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
        binary_difference = cv2.morphologyEx(binary_difference, cv2.MORPH_CLOSE, kernel)

        # Find contours
        contours, _ = cv2.findContours(binary_difference, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Fill the contours to create a mask
        mask_filled = np.zeros_like(binary_difference)
        cv2.drawContours(mask_filled, contours, -1, color=255, thickness=cv2.FILLED)

        # Expand the filled mask (dilation)
        mask_filled = cv2.dilate(mask_filled, kernel, iterations=1)

        return mask_filled  # Return mask as numpy array
    except Exception as e:
        print(f"Mask creation failed for input: {input_path}, error: {e}")
        return np.zeros((512, 512), dtype=np.uint8)


def apply_existing_mask(image, mask):
    """
    주어진 마스크를 이용해 이미지를 손상시킵니다.

    Parameters:
    - image: numpy array, 원본 이미지
    - mask: numpy array, 0과 1로 이루어진 마스크 (1: 손상 영역, 0: 유지 영역)

    Returns:
    - damaged_image: numpy array, 손상된 이미지
    """
    damaged_image = image.copy()
    damaged_image[mask == 255] = 0
    return damaged_image


def process_images_in_folder(input_folder, output_folder, damage_folder):
    """
    폴더에 있는 모든 이미지를 손상시키고 결과를 저장합니다.

    Parameters:
    - input_folder: str, 입력 이미지 폴더 경로
    - output_folder: str, 손상된 이미지 저장 경로
    - damage_folder: str, 손상 이미지를 생성할 기준 폴더 경로
    """
    # 출력 폴더가 없으면 생성
    os.makedirs(output_folder, exist_ok=True)

    # 입력 폴더의 이미지 처리
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, filename)
            damage_path = os.path.join(damage_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # 이미지 로드
            image = cv2.imread(input_path)
            if image is None:
                print(f"Skipping {filename}: unable to read file.")
                continue

            # 마스크 생성
            mask = mask_function(damage_path, input_path)

            # 마스크 적용
            damaged_image = apply_existing_mask(image, mask)

            # 결과 저장 (손상된 파일명 동일하게 저장)
            cv2.imwrite(output_path, damaged_image)
            print(f"Processed and saved: {output_path}")


# 사용 예제
input_folder = "train_gt"  # 원본 이미지 폴더 경로
damage_folder = "train_input"  # 손상 기준 폴더 경로
output_folder = "damage_images"  # 손상된 이미지 저장 폴더 경로

process_images_in_folder(input_folder, output_folder, damage_folder)


Processed and saved: damage_images/TRAIN_05927.png
Processed and saved: damage_images/TRAIN_05231.png
Processed and saved: damage_images/TRAIN_03153.png
Processed and saved: damage_images/TRAIN_01508.png
Processed and saved: damage_images/TRAIN_03591.png
Processed and saved: damage_images/TRAIN_07516.png
Processed and saved: damage_images/TRAIN_03792.png
Processed and saved: damage_images/TRAIN_20634.png
Processed and saved: damage_images/TRAIN_10974.png
Processed and saved: damage_images/TRAIN_11022.png
Processed and saved: damage_images/TRAIN_05396.png
Processed and saved: damage_images/TRAIN_15046.png
Processed and saved: damage_images/TRAIN_14762.png
Processed and saved: damage_images/TRAIN_28647.png
Processed and saved: damage_images/TRAIN_18872.png
Processed and saved: damage_images/TRAIN_11765.png
Processed and saved: damage_images/TRAIN_08101.png
Processed and saved: damage_images/TRAIN_13683.png
Processed and saved: damage_images/TRAIN_28343.png
Processed and saved: damage_ima